                                    **Arabic Boem Genrator**
                               
                             Generating Arabic poems using Markov chains.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#install requirment

!pip install pickle-mixin
!pip install markovify

In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_csv("../input/arabic-poetry/Arabic_poetry_dataset.csv", encoding="utf-8")
allowed_chars = ['ض', 'ص', 'ث', 'ق', 'ف', 'غ', 'ع', 'ه', 'خ', 'ح', 'ج', 'د', 'ش', 'س', 'ي', 'ب', 'ل', 'ا', 'ت', 'ن', 'م',
                 'ك', 'ط', 'ئ', 'ء', 'ؤ', 'ر' , 'ى', 'ة', 'و', 'ز', 'ظ', 'إ' ,'أ', 'آ', ' ', '\n', 'ذ', 'ّ']

def delete_puncs(poem):
    for l in poem:
        if l not in allowed_chars:
            poem = poem.replace(l, "")
    return poem

def extract_poems(poet_name):
    with open('.\input\{}.txt'.format(poet_name), 'w', encoding="utf-8") as f:
        for i in range(len(df)):
            if df['poet_name'][i].format(poet_name) == poet_name:
                poem = delete_puncs(df['poem_text'][i])
                f.write("{}\n\n".format(poem))
                
                

In [ ]:
all_poets = set()
for poet_name in df['poet_name']:
    all_poets.add(poet_name)

In [ ]:
import pickle

with open('../input/arabic-vocab1/vocabs.pkl', 'rb') as pickle_load:
    voc_list = pickle.load(pickle_load)
    

allowed_chars = ['ذ', 'ض', 'ص', 'ث', 'ق', 'ف', 'غ', 'ع', 'ه', 'خ', 'ح', 'ج', 'د',
                 'ش', 'س', 'ي', 'ب', 'ل', 'ا', 'أ', 'ت', 'ن', 'م', 'ك', 'ط', 'ئ', 'ء', 'ؤ', 'ر', 'ى', 
                 'ة', 'و', 'ز', 'ظ', 'ّ', ' ']

max_word_length = 9

In [ ]:
#extract all words on the same rythmes

def rhymes_with(word):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله']:
        word = word.replace('ّ', '')
    ending = word[-2:]
    rhymes = []
    for w in voc_list:
        if len(w) < max_word_length and w.endswith(ending):
            rhymes.append(w)
    return rhymes

def rhymes_with_last_n_chars(word, n):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله', 'فالله']:
        word = word.replace('ّ', '')
    ending = word[-n:]
    rhymes = []
    for w in voc_list:
        if len(w) < max_word_length and w.endswith(ending):
            rhymes.append(w)
    return rhymes

In [ ]:
import markovify
from tqdm import tqdm

def markov(text_file):
    with open(text_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text_model = markovify.NewlineText(text)
    return text_model

In [ ]:
def generate_poem_single_rhyme(poet_name, rhyme, iterations=3000, use_tqdm=False):
    n_of_rhyme_letters = len(rhyme)
    input_ff =  df.loc[df['poet_name'] == poet_name]
    input_file = '../input/poem-txt/.txt'
    text_model = markov(input_file)
    rhymes_list = rhymes_with_last_n_chars(rhyme, n_of_rhyme_letters)
    bayts = set()
    used_rhymes = set()
    
    poem = ""
    
    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)
        
    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]
        if (last_word in rhymes_list) and (last_word not in used_rhymes) and (bayt not in bayts):
            bayts.add(bayt)
            used_rhymes.add(last_word)
            poem += "{}\n".format(bayt)
            if not use_tqdm:
                print(bayt)
    return poem

def generate_poem_2_rhymes(poet_name, rhyme_1, rhyme_2, iterations=3000, use_tqdm=False):
    n_of_rhyme_1_letters = len(rhyme_1)
    n_of_rhyme_2_letters = len(rhyme_2)
    
    input_ff =  df.loc[df['poet_name'] == poet_name]
    input_file = '../input/poem-txt/.txt'
    text_model = markov(input_file)
    
    rhymes_1_list = rhymes_with_last_n_chars(rhyme_1, n_of_rhyme_1_letters)
    rhymes_2_list = rhymes_with_last_n_chars(rhyme_2, n_of_rhyme_2_letters)
    
    bayts_1 = set()
    bayts_2 = set()
    
    used_rhymes_1 = set()
    used_rhymes_2 = set()
    
    poem = ""
    
    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)
        
    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]
        
        if (last_word in rhymes_1_list) and (last_word not in used_rhymes_1) and (bayt not in bayts_1):
            bayts_1.add(bayt)
            used_rhymes_1.add(last_word)

        if (last_word in rhymes_2_list) and (last_word not in used_rhymes_2) and (bayt not in bayts_2):
            bayts_2.add(bayt)
            used_rhymes_2.add(last_word)                
    
    len_of_poem = min(len(bayts_1), len(bayts_2))
    for i in range(len_of_poem):
        poem += "{}\n{}\n".format(list(bayts_1)[i], list(bayts_2)[i])
        
    return poem

In [ ]:
x = generate_poem_single_rhyme(
                            'خلفان بن مصبح',
                            'الا',
                            iterations=3000,
                            use_tqdm=False
                            )